In [0]:
#pySpark Datatype is used when schema cannot be defined using list or string
#Data types types: struct,map,array

In [0]:
import findspark

In [0]:
findspark.init()

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName("RDDExample").getOrCreate()

In [0]:
from pyspark.sql.types import *

In [0]:
data=[(1,'a',30),(2,'b',32)]
user_schema=StructType([StructField("id",IntegerType()),
                        StructField("name",StringType()),
                        StructField("age",IntegerType())
                       ])
df=spark.createDataFrame(data,user_schema)

In [0]:
df.show()

In [0]:
data2= [
    (1, "Alice", ["Reading", "Hiking"]),
    (2, "Bob", ["Swimming", "Gardening", "Painting"]),
    (3, "Charlie", ["Cooking"]),
    (4, "David", ["Photography", "Skiing", "Cooking"])
]
user_schema2=StructType([StructField("id",IntegerType()),
                        StructField("name",StringType()),
                        StructField("hobbies",ArrayType(StringType()))
                       ])
df2=spark.createDataFrame(data2,user_schema2)
df2.show()

In [0]:
#get output as 1,Alice,Reading;1,Alice,hiking; 2,bob,swimming etc
from pyspark.sql.functions import *

In [0]:
df2.select("id","name",explode("hobbies")).show()

In [0]:
df2.withColumn("newhobby",explode("hobbies")).show()#creates new derived column


In [0]:
df2.withColumn("hobbies",explode("hobbies")).show()#replaces existing

In [0]:
df3=df2.withColumn("ingestion_date",current_timestamp()).show(truncate=False)#try truncate=True ans see what happens